In [63]:
import pandas as pd
import scipy.stats as ss

Statement: Company XYZ has started a new referral program on Oct, 31. Each user who refers a new user will get 10$ in credit when the new user buys something.
The program has been running for almost a month and the Growth Product Manager wants to know if it's been successful. She is very excited cause, since the referral program started, the company saw a spike in number of users and wants you to be able to give her some data she can show to her boss.

Questions: 
1. Can you estimate the impact the program had on the site?
2. Based on the data, what would you suggest to do as a next step?
3. The referral program wasn't really tested in a rigorous way. It simply started on a given day for all users and you are drawing conclusions by looking at the data before and after the test started. What kinds of risks this approach presents? Can you think of a better way to test the referral program and measure its impact?

In [51]:
data=pd.read_csv('/DS take home challenge/referral.csv')
data.head()

,user_id,date,country,money_spent,is_referral,device_id
0,2,2015-10-03,FR,65,0,EVDCJTZMVMJDG
1,3,2015-10-03,CA,54,0,WUBZFTVKXGQQX
2,6,2015-10-03,FR,35,0,CBAPCJRTFNUJG
3,7,2015-10-03,UK,73,0,PRGXJZAJKMXRH
4,7,2015-10-03,MX,35,0,PRGXJZAJKMXRH


In [52]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97341 entries, 0 to 97340
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   user_id      97341 non-null  int64 
 1   date         97341 non-null  object
 2   country      97341 non-null  object
 3   money_spent  97341 non-null  int64 
 4   is_referral  97341 non-null  int64 
 5   device_id    97341 non-null  object
dtypes: int64(3), object(3)
memory usage: 4.5+ MB


In [53]:
data['date']=pd.to_datetime(data['date'])

In [54]:
data.describe(include='all')

/var/folders/zt/hfwnrnh110q_vcpvn0nwl3qc0000gn/T/ipykernel_83629/1336221646.py:1: FutureWarning: Treating datetime data as categorical rather than numeric in `.describe` is deprecated and will be removed in a future version of pandas. Specify `datetime_is_numeric=True` to silence this warning and adopt the future behavior now.
  data.describe(include='all')


,user_id,date,country,money_spent,is_referral,device_id
count,97341.000000,97341,97341,97341.000000,97341.000000,97341
unique,NaN,56,9,NaN,NaN,17887
top,NaN,2015-11-14 00:00:00,UK,NaN,NaN,JOVUEUUQPQVXO
freq,NaN,3303,15493,NaN,NaN,35
first,NaN,2015-10-03 00:00:00,NaN,NaN,NaN,NaN
last,NaN,2015-11-27 00:00:00,NaN,NaN,NaN,NaN
mean,6354.801029,NaN,NaN,44.692360,0.287823,NaN
std,5682.991186,NaN,NaN,22.791839,0.452751,NaN
min,1.000000,NaN,NaN,10.000000,0.000000,NaN
25%,2020.000000,NaN,NaN,27.000000,0.000000,NaN


User referal program starts right in the middle betwwen 2015-10-03 and 2015-11-27.

In [55]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 97341 entries, 0 to 97340
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   user_id      97341 non-null  int64         
 1   date         97341 non-null  datetime64[ns]
 2   country      97341 non-null  object        
 3   money_spent  97341 non-null  int64         
 4   is_referral  97341 non-null  int64         
 5   device_id    97341 non-null  object        
dtypes: datetime64[ns](1), int64(3), object(2)
memory usage: 4.5+ MB


### Question 1

In [81]:
def stats(df):
    stats=df.groupby('date').agg({'user_id':lambda x:x.nunique(),'date':'count','money_spent':'sum'}).rename(columns={'user_id':'n_user','date':'n_purchase','money_spent':'total_spent'})
    pre_data=stats[stats.index<'2015-10-31']
    post_data=stats[stats.index>='2015-10-31']
    pre_mean=pre_data.mean()
    post_mean=post_data.mean()
    colnames=['total_spent','n_purchase','n_user']
    df_stats=pd.DataFrame({'mean_pre':pre_mean,'mean_post':post_mean,'mean_diff':post_mean-pre_mean},index=colnames)
    df_stats['pvalue']=ss.ttest_ind(pre_data,post_data,equal_var=False).pvalue
    return df_stats


In [82]:
stats(data)

,mean_pre,mean_post,mean_diff,pvalue
total_spent,71657.000000,83714.392857,12057.392857,0.119091
n_purchase,1690.750000,1785.714286,94.964286,0.696514
n_user,1384.464286,1686.964286,302.500000,0.270388


Per the summary above, none of the increment are significant at 5% significance level. This is for the whole country. We will dive into each country level below. 

In [83]:
data['country'].value_counts()

UK    15493
FR    15396
US    15280
IT    11446
DE    11093
ES     9831
CA     9440
MX     8133
CH     1229
Name: country, dtype: int64

In [84]:
data.groupby('country').apply(stats)

mean_pre     mean_post    mean_diff    pvalue
country                                                               
CA      total_spent   7468.428571   7880.428571   412.000000  0.536511
        n_purchase     177.142857    160.000000   -17.142857  0.467970
        n_user         173.285714    159.178571   -14.107143  0.703409
CH      total_spent   1536.321429   1023.892857  -512.428571  0.006285
        n_purchase      26.821429     17.071429    -9.750000  0.006145
        n_user          26.714286     17.071429    -9.642857  0.013882
DE      total_spent   9856.750000   8013.964286 -1842.785714  0.031330
        n_purchase     232.142857    164.035714   -68.107143  0.023596
        n_user         224.964286    163.250000   -61.714286  0.162919
ES      total_spent   6648.642857   8660.571429  2011.928571  0.115907
        n_purchase     156.607143    194.500000    37.892857  0.145276
        n_user         153.392857    193.214286    39.821429  0.075043
FR      total_spent  10385.250000  13635.000000  3249.750000  0.082247
        n_purchase     244.142857    305.714286    61.571429  0.117991
        n_user         236.500000    302.535714    66.035714  0.063686
IT      total_spent   7651.571429  10193.428571  2541.857143  0.087821
        n_purchase     180.857143    227.928571    47.071429  0.114909
        n_user         176.535714    226.107143    49.571429  0.051460
MX      total_spent   4975.464286   7033.214286  2057.750000  0.052406
        n_purchase     126.464286    164.000000    37.535714  0.064861
        n_user         124.392857    163.107143    38.714286  0.019340
UK      total_spent  11213.535714  14196.428571  2982.892857  0.408796
        n_purchase     264.285714    289.035714    24.750000  0.522367
        n_user         255.571429    286.321429    30.750000  0.096980
US      total_spent  11921.035714  13077.464286  1156.428571  0.737720
        n_purchase     282.285714    263.428571   -18.857143  0.615601
        n_user         273.178571    261.107143   -12.071429  0.497749

1. daily spent in 'CH' and 'DE' are significantly decreased.
2. 'MX','IT','FR','ES','UK', their daily spent are significant increased.
3. There are other sales increases in some countries, but not significant. 

### Question 2

I will suggest to perform A/B test to study the impact of the program. Also research on why there are such difference in those countries. 

### Question 3

 1. "User Referral" program isn't the only difference between control group and test group. for example, there may be other factors e.g. holiday. 
 2. A/B test 
 randomly split the customers into two groups and let only one group expose to the user referral program. Run the experiement at least 1-2 business weeks. The primary metrics include the number of daily customers, daily transactions. 
 3. We can also use observational studies (e.g. propensity scoring matching) if it's diffcult to perform randomization. 
 